In [0]:
import pandas as pd
import numpy as np

In [0]:
def clean_estimate(estimate):
  """
  takes the estimate column, adds low estimate, high estimate, and currency
  columns to the dataframe and drops the original estimate column.
  :param: estimate = list (df column)
  returns: none, adds the columns and drop the column when called
  """
  low = []
  high = []
  currency = []

  for i in range(len(estimate)):
    est = estimate[i]
    try:
      temp = est.split()
      if len(temp) == 4:
        low.append(temp[0])
        high.append(temp[2])
        currency.append(temp[3])
      elif len(temp) == 2:
        low.append(temp[0])
        high.append(np.nan)
        currency.append(temp[1])
      else:
        low.append(np.nan)
        high.append(np.nan)
        currency.append(np.nan)
    except:
      pass
    
  auction_history['Low Estimate'] = low
  auction_history['High Estimate'] = high
  auction_history['Currency'] = currency
  auction_history.drop(['Estimate_Range'], axis=1, inplace=True)
  return

def clean_realized_price(prices):
  """
  takes realized prices from df and removes currency leaving just the numeric value
  :param: prices = list, df column realized_price
  return: list
  """
  for i in range(len(prices)):
    try:
      temp = prices[i].split()
      if len(temp) < 2:
        prices[i] = -1
      else:
        prices[i] = temp[0]
    except:
      pass
  
  return prices

In [0]:
auction_history = pd.read_csv('https://raw.githubusercontent.com/AndrewMarksArt/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/master/Damien_Hirst_Auction_Results.csv')
auction_history.drop(['ID_Batch_Loop#', 'Artist_Name', 'Date_Created', 'Size'], axis=1, inplace=True)
auction_history = auction_history.rename(columns={'Sale_Date': 'Date'})
auction_history.head()

In [0]:
auction_history.shape

In [0]:
fx = pd.read_csv('https://raw.githubusercontent.com/AndrewMarksArt/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/master/additional_fx_data.csv')
fx.head()

In [0]:
clean_estimate(auction_history['Estimate_Range'])
auction_history['Realized_Price'] = clean_realized_price(auction_history['Realized_Price'])

In [0]:
auction_history.head()

In [0]:
for i in range(len(auction_history['Currency'])):
  try:
    if '*' in auction_history['Currency'][i]:
      auction_history['Currency'][i] = 'EUR'
    if 'Sold' in auction_history['Currency'][i]:
      auction_history['Currency'][i] = np.nan
  except:
    pass

In [0]:
auction_history['Currency'].fillna('USD', inplace=True)

In [0]:
country_dict = {'GBP': 'United Kingdom', 'USD': 'United States', 'EUR': 'Europe', 'HKD': 'Hong Kong', 'CNY': 'China', 'CHF': 'Switzerland',
             'SEK': 'Sweden', 'JPY': 'Japan', 'KRW': 'South Korea', 'AUD': 'Australia', 'NOK': 'Norway', 'ZAR': 'South Africa', 'DKK': 'Denmark', 'NZD': 'New Zealand', 
               'TWD': 'Taiwan', 'SGD': 'Singapore', 'MXN': 'Mexico', 'CZK': 'Czech Republic'}

In [0]:
country = []
currency = auction_history['Currency']

for i in range(len(currency)):
  try:
    country.append(country_dict.get(currency[i]))
  except:
    pass
    
auction_history['Country'] = country

In [0]:
sale = auction_history['Date']
for i in range(len(sale)):
  try:
    if 'no' in sale[i]:
      sale[i] = np.nan
  except:
    pass

In [0]:
auction_history['Date'] = pd.to_datetime(auction_history['Date'])

In [0]:
price = []

for i in range(len(auction_history['Realized_Price'])):
  try:
    if 'Not' in auction_history['Realized_Price'][i]:
      price.append(0)
    elif 'not' in auction_history['Realized_Price'][i]:
      price.append(0)
    else:
      price.append(auction_history['Realized_Price'][i])
  except:
    price.append(0)

auction_history['Realized_Price'] = price

In [0]:
auction_history['Realized_Price'] = auction_history['Realized_Price'].str.replace(',','')
auction_history['Low Estimate'] = auction_history['Low Estimate'].str.replace(',','')
auction_history['High Estimate'] = auction_history['High Estimate'].str.replace(',','')

In [0]:
auction_history['Realized_Price'] = auction_history['Realized_Price'].fillna(0)
auction_history = auction_history.dropna(subset=['Low Estimate'])
low = auction_history['Low Estimate'].where(auction_history['High Estimate'].isnull()).dropna()
auction_history['High Estimate'] = auction_history['High Estimate'].fillna(low)

In [0]:
house = []
temp = auction_history['Auction_House']

for i in range(len(temp)):
  try:
    if "Sotheby's" in temp[i]:
      house.append("Sotheby's")
    elif "Christie's" in temp[i]:
      house.append("Christie's")
    elif "Phillips" in temp[i]:
      house.append("Phillips")
    else:
      house.append("Other")
  except:
    house.append("Other")
    
auction_history['House'] = house

In [0]:
auction_history = auction_history.dropna(subset=['Date'])

In [0]:
auction_history.head()

In [0]:
auction_history.shape

In [0]:
fx = fx.dropna(subset=['Value']).reset_index()
fx = fx.rename(columns={'Value': 'FX Rate'})
fx = fx.drop('index',axis=1)
fx.head()

In [0]:
fx['Date'] = pd.to_datetime(fx['Date'])

In [0]:
merged = pd.merge(auction_history, fx, on=['Date','Country'], how='left')
merged.head()

In [0]:
merged.shape

In [0]:
merged['FX Rate'].isnull().sum()

In [0]:
usa = merged[merged['Country'] == 'United States']
usa['FX Rate'] = usa['FX Rate'].fillna(1)

not_usa = merged[merged['Country'] != 'United States']

merged = pd.concat([usa, not_usa])

In [0]:
merged.isnull().sum()

In [0]:
merged['FX Rate'].fillna(method='ffill')